### Import Modules

In [110]:
import numpy as np
from CoolProp.CoolProp import PropsSI


### Specify Parameters

In [111]:
# tsp=thermosyphon
Nt=9
l_e= 0.45 # evaporator tsp len
l_a=0.1 # adiabatic tsp len
l_c=0.45 # condenser tsp len
tsp_od=9.5/1000
tsp_thick=1/1000
PT=15/1000 #tube pitch
F=70/100 #filling ratio


B=86/1000 #baffle spacing
shell_inlet_dia=25.4/1000
shell_id=99.6/1000

m = 4/60 # L/min to kg/s
Ti_e=90 #°C
Ti_c=20 #°C
To_e=80 # assume
To_c=30 # assume
q_hx=5000 # assume
q_e=q_hx #heat flux
q_c=q_hx
k_ss=16.2 # W/mK
k_ms=50 

### Calculate Primary parameters

In [112]:
tsp_id=tsp_od-tsp_thick*2
Dhs=4*PT**2/(np.pi*tsp_od)-tsp_od #hydraulic dia
Ds=shell_id #Shell inside dia
C=PT-tsp_od #Clearance
As=Ds*C*B/PT #Bundle cross flow area


### Initial guess of vapour temperature

In [113]:
Cp_e = PropsSI('C', 'T', Ti_e + 273.15, 'P', 101325, 'Water')  # Specific heat capacity (J/(kg·K))
Cp_c = PropsSI('C', 'T', Ti_c + 273.15, 'P', 101325, 'Water')  # Specific heat capacity (J/(kg·K))

Tv=(Ti_e*Cp_e+Ti_c*Cp_c)/(Cp_e+Cp_c)
print(f"Vapor Temperature, Tv = {Tv}")

Vapor Temperature, Tv = 55.08825738644433


### Evaporator Shell Hot Fluid

In [114]:
fluid = "Water"
temperature = (Ti_e+To_e)/2 + 273.15  # Convert °C to K
pressure = 101325  # Pressure in Pa (1 atm)

#properties
rho_e = PropsSI('D', 'T', temperature, 'P', pressure, fluid)  # Density in kg/m³
mu_e = PropsSI('V', 'T', temperature, 'P', pressure, fluid)  # Dynamic viscosity (Pa·s)
Cp_e = PropsSI('C', 'T', temperature, 'P', pressure, fluid)  # Specific heat capacity (J/(kg·K))
k_e = PropsSI('L', 'T', temperature, 'P', pressure, fluid)  # Thermal conductivity (W/(m·K))
Pr_e=PropsSI('PRANDTL', 'T', temperature, 'P', pressure, fluid) # Prandtl Number

print(f"Properites of {fluid} at {Ti_e} °C:")
print(f"\tρ_l = {rho_e} kg/m³")
print(f"\tμ = {mu_e} Pa·s")
print(f"\tPr = {Pr_e}")
print(f"\tk = {k_e}")


Properites of Water at 90 °C:
	ρ_l = 968.6114401082684 kg/m³
	μ = 0.0003330754563272273 Pa·s
	Pr = 2.088096210424027
	k = 0.6700671488300696


In [115]:
Vs=m/(rho_e*As)
print(f"Hot Shell fluid Velocity, Vs = {Vs}")

Re_s=Vs*Dhs*rho_e/mu_e
print(f"Renolds Number, Re_s = {Re_s}")

Nu = 0.36*pow(Re_s,0.55)*pow(Pr_e,1/3)
print(f"Nusselt Number, Nu = {Nu}")

#coefficient of external heat transer over evaporator
h_ex_e=Nu*k_e/Dhs
print(f"Heat transfer coefficient, h_ex_e = {h_ex_e}")
A_e=np.pi*tsp_od*l_e*Nt
# Convection resistance over evaporator
R1=1/(h_ex_e*A_e)
print(f"Forced convection resistance over Evaporator, R1 = {R1}")

r_e=tsp_od/2
r_i=r_e-tsp_thick
R2=np.log(r_e/r_i)/(2*np.pi*k_ss*l_e)
print(f"Radial heat conduction resistance through Evaporator wall, R2 = {R2}")

Hot Shell fluid Velocity, Vs = 0.02191441726863351
Renolds Number, Re_s = 1316.3647615642558
Nusselt Number, Nu = 23.907983027994955
Heat transfer coefficient, h_ex_e = 775.5716160857919
Forced convection resistance over Evaporator, R1 = 0.010667179229727514
Radial heat conduction resistance through Evaporator wall, R2 = 0.005160828877966428


### Evaporator Thermosyphon


In [116]:
#thermosyphon

fluid = "Water"
temperature = Tv + 273.15  # Convert °C to K
pressure = 101325  # Pressure in Pa (1 atm)

#properties
rho_l_v = PropsSI('D', 'T', temperature, 'P', pressure, fluid)  # Density in kg/m³
rho_v_v = PropsSI('D', 'T', temperature, 'Q', 1, fluid)  # Density in kg/m³ (Q=1 means saturated vapor)
mu_l_v = PropsSI('V', 'T', temperature, 'P', pressure, fluid)  # Dynamic viscosity (Pa·s)
Cp_l_v = PropsSI('C', 'T', temperature, 'P', pressure, fluid)  # Specific heat capacity (J/(kg·K))
k_v = PropsSI('L', 'T', temperature, 'P', pressure, fluid)  # Thermal conductivity (W/(m·K))
Pr_v=PropsSI('PRANDTL', 'T', temperature, 'P', pressure, fluid) # Prandtl Number
P_v = PropsSI('P', 'T', temperature, 'Q', 0, fluid)  # 'Q=0' means saturated liquid
h_lv = PropsSI('H', 'T', temperature, 'Q', 1, fluid) - PropsSI('H', 'T', temperature, 'Q', 0, fluid)



print(f"Properites of {fluid} at {Tv} °C:")
print(f"\tρ_l = {rho_l_v} kg/m³")
print(f"\tρ_v = {rho_v_v} kg/m³")
print(f"\tμ = {mu_l_v} Pa·s")
print(f"\tPr = {Pr_v}")
print(f"\tk = {k_v} W/mK")
print(f"Vapor Pressure, Pv = {P_v} Pa")
print(f"Latent heat of vaporization, h_lv = {h_lv} J/kg")


Properites of Water at 55.08825738644433 °C:
	ρ_l = 985.6503289489771 kg/m³
	ρ_v = 0.10497172066517207 kg/m³
	μ = 0.0005029178905094976 Pa·s
	Pr = 3.2559358830523197
	k = 0.6461121543366464 W/mK
Vapor Pressure, Pv = 15828.870700067382 Pa
Latent heat of vaporization, h_lv = 2369623.4119260954 J/kg


In [117]:
phi2=pow(h_lv*k_v*(rho_l_v**0.5)/mu_l_v,0.25)*(pow(rho_l_v,0.65)*pow(k_v,0.3)*pow(Cp_l_v,0.7))/(pow(rho_v_v,0.25)*pow(h_lv,0.4)*pow(mu_l_v,0.7))
phi3=pow(P_v/101325,0.23)*(pow(rho_l_v,0.65)*pow(k_v,0.3)*pow(Cp_l_v,0.7))/(pow(rho_v_v,0.25)*pow(h_lv,0.4)*pow(mu_l_v,0.7))
#thermal resistance associated to pool boiling
R3p=1/((9.81**0.2)*phi3*(q_e**0.4)*pow(np.pi*tsp_id*l_e,0.6))
#thermal resistance associated to falling film located above the evaporator pool
R3f=0.345*pow(q_e,1/3)/(pow(tsp_id,4/3)*pow(9.81,1/3)*l_e*pow(phi2,4/3))
R3=R3p*F+(1-F)*R3f
print(f"Boiling thermal resistance, R3 = {R3}")
R_e=R1+R2+R3
print(f"Evaporator Resistance, R_e = {R_e}")

Boiling thermal resistance, R3 = 1.3209187476402971e-05
Evaporator Resistance, R_e = 0.015841217295170344


### Evaluate_R_e Function

In [118]:
def Evaluate_R_e(Ti_e, To_e):
    #--------------------shell---------------
    fluid = "Water"
    temperature = (Ti_e+To_e)/2 + 273.15  # Convert °C to K
    pressure = 101325  # Pressure in Pa (1 atm)

    #properties
    rho_e = PropsSI('D', 'T', temperature, 'P', pressure, fluid)  # Density in kg/m³
    mu_e = PropsSI('V', 'T', temperature, 'P', pressure, fluid)  # Dynamic viscosity (Pa·s)
    Cp_e = PropsSI('C', 'T', temperature, 'P', pressure, fluid)  # Specific heat capacity (J/(kg·K))
    k_e = PropsSI('L', 'T', temperature, 'P', pressure, fluid)  # Thermal conductivity (W/(m·K))
    Pr_e=PropsSI('PRANDTL', 'T', temperature, 'P', pressure, fluid) # Prandtl Number

    # print(f"Properites of {fluid} at {Ti_e} °C:")
    # print(f"\tρ_l = {rho_e} kg/m³")
    # print(f"\tμ = {mu_e} Pa·s")
    # print(f"\tPr = {Pr_e}")
    # print(f"\tk = {k_e}")

    Vs=m/(rho_e*As)
    # print(f"Hot Shell fluid Velocity, Vs = {Vs}")

    Re_s=Vs*Dhs*rho_e/mu_e
    # print(f"Renolds Number, Re_s = {Re_s}")

    Nu = 0.36*pow(Re_s,0.55)*pow(Pr_e,1/3)
    # print(f"Nusselt Number, Nu = {Nu}")

    #coefficient of external heat transer over evaporator
    h_ex_e=Nu*k_e/Dhs
    # print(f"Heat transfer coefficient, h_ex_e = {h_ex_e}")
    A_e=np.pi*tsp_od*l_e*Nt
    # Convection resistance over evaporator
    R1=1/(h_ex_e*A_e)
    # print(f"Forced convection resistance over Evaporator, R1 = {R1}")

    r_e=tsp_od/2
    r_i=r_e-tsp_thick
    R2=np.log(r_e/r_i)/(2*np.pi*k_ss*l_e)
    # print(f"Radial heat conduction resistance through Evaporator wall, R2 = {R2}")


    #----------------------thermosyphon------------------
    fluid = "Water"
    temperature = Tv + 273.15  # Convert °C to K
    pressure = 101325  # Pressure in Pa (1 atm)

    #properties
    rho_l_v = PropsSI('D', 'T', temperature, 'P', pressure, fluid)  # Density in kg/m³
    rho_v_v = PropsSI('D', 'T', temperature, 'Q', 1, fluid)  # Density in kg/m³ (Q=1 means saturated vapor)
    mu_l_v = PropsSI('V', 'T', temperature, 'P', pressure, fluid)  # Dynamic viscosity (Pa·s)
    Cp_l_v = PropsSI('C', 'T', temperature, 'P', pressure, fluid)  # Specific heat capacity (J/(kg·K))
    k_v = PropsSI('L', 'T', temperature, 'P', pressure, fluid)  # Thermal conductivity (W/(m·K))
    Pr_v=PropsSI('PRANDTL', 'T', temperature, 'P', pressure, fluid) # Prandtl Number
    P_v = PropsSI('P', 'T', temperature, 'Q', 0, fluid)  # 'Q=0' means saturated liquid
    h_lv = PropsSI('H', 'T', temperature, 'Q', 1, fluid) - PropsSI('H', 'T', temperature, 'Q', 0, fluid)

    # print(f"Properites of {fluid} at {Tv} °C:")
    # print(f"\tρ_l = {rho_l_v} kg/m³")
    # print(f"\tρ_v = {rho_v_v} kg/m³")
    # print(f"\tμ = {mu_l_v} Pa·s")
    # print(f"\tPr = {Pr_v}")
    # print(f"\tk = {k_v} W/mK")
    # print(f"Vapor Pressure, Pv = {P_v} Pa")
    # print(f"Latent heat of vaporization, h_lv = {h_lv} J/kg")

    phi2=pow(h_lv*k_v*(rho_l_v**0.5)/mu_l_v,0.25)*(pow(rho_l_v,0.65)*pow(k_v,0.3)*pow(Cp_l_v,0.7))/(pow(rho_v_v,0.25)*pow(h_lv,0.4)*pow(mu_l_v,0.7))
    phi3=pow(P_v/101325,0.23)*(pow(rho_l_v,0.65)*pow(k_v,0.3)*pow(Cp_l_v,0.7))/(pow(rho_v_v,0.25)*pow(h_lv,0.4)*pow(mu_l_v,0.7))
    #thermal resistance associated to pool boiling
    R3p=1/((9.81**0.2)*phi3*(q_e**0.4)*pow(np.pi*tsp_id*l_e,0.6))
    #thermal resistance associated to falling film located above the evaporator pool
    R3f=0.345*pow(q_e,1/3)/(pow(tsp_id,4/3)*pow(9.81,1/3)*l_e*pow(phi2,4/3))
    R3=R3p*F+(1-F)*R3f
    # print(f"Boiling thermal resistance, R3 = {R3}")
    R_e=R1+R2+R3
    # print(f"Evaporator Resistance, R_e = {R_e}")
    return R_e
    

### Condenser Shell Cold Fluid

In [119]:
fluid = "Water"
temperature = (Ti_c+To_c)/2 + 273.15  # Convert °C to K
pressure = 101325  # Pressure in Pa (1 atm)

#properties
rho_l_c = PropsSI('D', 'T', temperature, 'P', pressure, fluid)  # Density in kg/m³
rho_v_c = PropsSI('D', 'T', temperature, 'Q', 1, fluid)  # Density in kg/m³ (Q=1 means saturated vapor)
mu_c = PropsSI('V', 'T', temperature, 'P', pressure, fluid)  # Dynamic viscosity (Pa·s)
Cp_c = PropsSI('C', 'T', temperature, 'P', pressure, fluid)  # Specific heat capacity (J/(kg·K))
k_c = PropsSI('L', 'T', temperature, 'P', pressure, fluid)  # Thermal conductivity (W/(m·K))
Pr_c=PropsSI('PRANDTL', 'T', temperature, 'P', pressure, fluid) # Prandtl Number
P_v_c = PropsSI('P', 'T', temperature, 'Q', 0, fluid)  # 'Q=0' means saturated liquid
h_lv_c = PropsSI('H', 'T', temperature, 'Q', 1, fluid) - PropsSI('H', 'T', temperature, 'Q', 0, fluid)



print(f"Properites of {fluid} at {Ti_c} °C:")
print(f"\tρ_l = {rho_l_c} kg/m³")
print(f"\tρ_v = {rho_v_c} kg/m³")
print(f"\tμ = {mu_c} Pa·s")
print(f"\tPr = {Pr_c}")
print(f"\tk = {k_c} W/mK")
print(f"Vapor Pressure, Pv = {P_v_c} Pa")
print(f"Latent heat of vaporization, h_lv = {h_lv_c} J/kg")

Properites of Water at 20 °C:
	ρ_l = 997.047636760347 kg/m³
	ρ_v = 0.02307480418276038 kg/m³
	μ = 0.0008900224890776964 Pa·s
	Pr = 6.135804963909522
	k = 0.6065160802197994 W/mK
Vapor Pressure, Pv = 3169.9293676106945 Pa
Latent heat of vaporization, h_lv = 2441676.1749126264 J/kg


In [120]:
Vs=m/(rho_l_c*As)
print(f"Condenser Shell Side Velocity, Vs = {Vs}")

Re_s=Vs*Dhs*rho_l_c/mu_c
print(f"Renolds Number, Re_s = {Re_s}")

Nu = 0.36*pow(Re_s,0.55)*pow(Pr_c,1/3)
print(f"Nusselt Number, Nu = {Nu}")

r_e=tsp_od/2
r_i=r_e-tsp_thick
R8=np.log(r_e/r_i)/(2*np.pi*k_ss*l_c)

#coefficient of external heat transer in evaporator
h_ex_c=Nu*k_c/Dhs
print(f"Convection Heat transfer coefficient, h_ex_c = {h_ex_c}")
A_c=np.pi*tsp_od*l_c*Nt
# Convection resistance in hot fluid
R9=1/(h_ex_c*A_c)
print(f"Forced convection resistance over Condenser, R9 = {R9}")
print(f"Radial heat conduction resistance through Condenser wall, R8 = {R8}")

Condenser Shell Side Velocity, Vs = 0.021289409339231683
Renolds Number, Re_s = 492.6266459912126
Nusselt Number, Nu = 19.943950794471043
Convection Heat transfer coefficient, h_ex_c = 585.6176465105275
Forced convection resistance over Condenser, R9 = 0.014127240672430517
Radial heat conduction resistance through Condenser wall, R8 = 0.005160828877966428


### Condenser Thermosyphon

In [121]:
phi2=pow(h_lv*k_v*(rho_l_v**0.5)/mu_l_v,0.25)*(pow(rho_l_v,0.65)*pow(k_v,0.3)*pow(Cp_l_v,0.7))/(pow(rho_v_v,0.25)*pow(h_lv,0.4)*pow(mu_l_v,0.7))
R7=0.345*pow(q_c,1/3)/(pow(tsp_id,4/3)*pow(9.81,1/3)*l_c*pow(phi2,4/3))
print(f"Condensation thermal resistance, R7 = {R7}")

R_c=R7+R8+R9
print(f"Condenser Resistance, R_c = {R_c}")

Condensation thermal resistance, R7 = 1.1385563128926466e-06
Condenser Resistance, R_c = 0.019289208106709838


In [122]:
R_total=R_e+R_c
print(f"Total thermal resistance, R_total = {R_total}")

Total thermal resistance, R_total = 0.03513042540188018


### Evaluate_R_c Function

In [123]:
def Evaluate_R_c(Ti_c, To_c):
    fluid = "Water"
    temperature = (Ti_c+To_c)/2 + 273.15  # Convert °C to K
    pressure = 101325  # Pressure in Pa (1 atm)

    #properties
    rho_l_c = PropsSI('D', 'T', temperature, 'P', pressure, fluid)  # Density in kg/m³
    rho_v_c = PropsSI('D', 'T', temperature, 'Q', 1, fluid)  # Density in kg/m³ (Q=1 means saturated vapor)
    mu_c = PropsSI('V', 'T', temperature, 'P', pressure, fluid)  # Dynamic viscosity (Pa·s)
    Cp_c = PropsSI('C', 'T', temperature, 'P', pressure, fluid)  # Specific heat capacity (J/(kg·K))
    k_c = PropsSI('L', 'T', temperature, 'P', pressure, fluid)  # Thermal conductivity (W/(m·K))
    Pr_c=PropsSI('PRANDTL', 'T', temperature, 'P', pressure, fluid) # Prandtl Number
    P_v_c = PropsSI('P', 'T', temperature, 'Q', 0, fluid)  # 'Q=0' means saturated liquid
    h_lv_c = PropsSI('H', 'T', temperature, 'Q', 1, fluid) - PropsSI('H', 'T', temperature, 'Q', 0, fluid)

    # print(f"Properites of {fluid} at {Ti_c} °C:")
    # print(f"\tρ_l = {rho_l_c} kg/m³")
    # print(f"\tρ_v = {rho_v_c} kg/m³")
    # print(f"\tμ = {mu_c} Pa·s")
    # print(f"\tPr = {Pr_c}")
    # print(f"\tk = {k_c} W/mK")
    # print(f"Vapor Pressure, Pv = {P_v_c} Pa")
    # print(f"Latent heat of vaporization, h_lv = {h_lv_c} J/kg")

    Vs=m/(rho_l_c*As)
    # print(f"Condenser Shell Side Velocity, Vs = {Vs}")

    Re_s=Vs*Dhs*rho_l_c/mu_c
    # print(f"Renolds Number, Re_s = {Re_s}")

    Nu = 0.36*pow(Re_s,0.55)*pow(Pr_c,1/3)
    # print(f"Nusselt Number, Nu = {Nu}")

    r_e=tsp_od/2
    r_i=r_e-tsp_thick
    R8=np.log(r_e/r_i)/(2*np.pi*k_ss*l_c)

    #coefficient of external heat transer in evaporator
    h_ex_c=Nu*k_c/Dhs
    # print(f"Convection Heat transfer coefficient, h_ex_c = {h_ex_c}")
    A_c=np.pi*tsp_od*l_c*Nt
    # Convection resistance in hot fluid
    R9=1/(h_ex_c*A_c)
    # print(f"Forced convection resistance over Condenser, R9 = {R9}")
    # print(f"Radial heat conduction resistance through Condenser wall, R8 = {R8}")


        #----------------------thermosyphon------------------
    fluid = "Water"
    temperature = Tv + 273.15  # Convert °C to K
    pressure = 101325  # Pressure in Pa (1 atm)

    #properties
    rho_l_v = PropsSI('D', 'T', temperature, 'P', pressure, fluid)  # Density in kg/m³
    rho_v_v = PropsSI('D', 'T', temperature, 'Q', 1, fluid)  # Density in kg/m³ (Q=1 means saturated vapor)
    mu_l_v = PropsSI('V', 'T', temperature, 'P', pressure, fluid)  # Dynamic viscosity (Pa·s)
    Cp_l_v = PropsSI('C', 'T', temperature, 'P', pressure, fluid)  # Specific heat capacity (J/(kg·K))
    k_v = PropsSI('L', 'T', temperature, 'P', pressure, fluid)  # Thermal conductivity (W/(m·K))
    Pr_v=PropsSI('PRANDTL', 'T', temperature, 'P', pressure, fluid) # Prandtl Number
    P_v = PropsSI('P', 'T', temperature, 'Q', 0, fluid)  # 'Q=0' means saturated liquid
    h_lv = PropsSI('H', 'T', temperature, 'Q', 1, fluid) - PropsSI('H', 'T', temperature, 'Q', 0, fluid)

    # print(f"Properites of {fluid} at {Tv} °C:")
    # print(f"\tρ_l = {rho_l_v} kg/m³")
    # print(f"\tρ_v = {rho_v_v} kg/m³")
    # print(f"\tμ = {mu_l_v} Pa·s")
    # print(f"\tPr = {Pr_v}")
    # print(f"\tk = {k_v} W/mK")
    # print(f"Vapor Pressure, Pv = {P_v} Pa")
    # print(f"Latent heat of vaporization, h_lv = {h_lv} J/kg")

    phi2=pow(h_lv*k_v*(rho_l_v**0.5)/mu_l_v,0.25)*(pow(rho_l_v,0.65)*pow(k_v,0.3)*pow(Cp_l_v,0.7))/(pow(rho_v_v,0.25)*pow(h_lv,0.4)*pow(mu_l_v,0.7))
    R7=0.345*pow(q_c,1/3)/(pow(tsp_id,4/3)*pow(9.81,1/3)*l_c*pow(phi2,4/3))
    # print(f"Condensation thermal resistance, R7 = {R7}")

    R_c=R7+R8+R9
    # print(f"Condenser Resistance, R_c = {R_c}")
    return R_c
    

### Optimization 

In [124]:
q_th=q_hx/Nt
Tavg_e=Tv-(((Tv-Ti_e)-(Tv-To_e))/np.log((Tv-Ti_e)/(Tv-To_e)))
Tavg_c=Tv-(((Tv-Ti_c)-(Tv-To_c))/np.log((Tv-Ti_c)/(Tv-To_c)))
q_e=(Tavg_e-Tv)/R_e
q_c=(Tv-Tavg_c)/R_c
print(f"q_e = {q_e}")
print(f"q_c = {q_c}")

q_e = 1870.5026166179407
q_c = 1545.3835452337464


In [125]:
#tolerance
alpha=abs(1-q_c/q_e)
alpha

0.1738137485057587

In [126]:
i=0
while(alpha>0.1/100 and i<50):
    To_e=Ti_e-q_hx/(m*Cp_e)
    To_c=Ti_c+q_hx/(m*Cp_c)
    # print(f"To_e = {To_e}")
    # print(f"To_c = {To_c}")
    
    R_e=Evaluate_R_e(Ti_e,To_e)
    R_c=Evaluate_R_c(Ti_c,To_c)
    R_total = R_e+R_c

    Tv=(R_c*Tavg_e+R_e*Tavg_c)/(R_e+R_c)
    # print(f"Tv = {Tv}")
    Tavg_e=Tv-(((Tv-Ti_e)-(Tv-To_e))/np.log((Tv-Ti_e)/(Tv-To_e)))
    Tavg_c=Tv-(((Tv-Ti_c)-(Tv-To_c))/np.log((Tv-Ti_c)/(Tv-To_c)))

    q_hx=(Tavg_e-Tavg_c)/R_total
    q_th=q_hx/Nt

    q_e=(Tavg_e-Tv)/R_e
    q_c=(Tv-Tavg_c)/R_c
    alpha=abs(1-q_c/q_e)
    print(f'iteration {i}: α = {alpha*100:.2f}%  q_hx = {q_hx:.2f} W')
    i+=1

iteration 0: α = 3.97%  q_hx = 1431.06 W
iteration 1: α = 5.28%  q_hx = 1834.50 W
iteration 2: α = 0.54%  q_hx = 1793.46 W
iteration 3: α = 0.06%  q_hx = 1797.68 W


In [127]:
To_e=Ti_e-q_hx/(m*Cp_e)
To_c=Ti_c+q_hx/(m*Cp_c)
print(f"Evaporator Outlet Temperature: {To_e:.2f}")
print(f"Condensor Outlet Temperature: {To_c:.2f}")

Evaporator Outlet Temperature: 83.58
Condensor Outlet Temperature: 26.45
